# PCA Algorithm
 - Step 1: Centre the data
 - Step 2: Compute the covariance matrix
 - Step 3: Compute the eigenvalues and eigenvectors of the covariance matrix
 - Step 4: Sort the eigenvectors by decreasing eigenvalues
 - Step 5: Choose the k eigenvectors that correspond to the k largest eigenvalues
 - Step 6: Construct a new matrix with the k eigenvectors
 - Step 7: Project the original data onto the new matrix
 - Step 8: Reconstruct the original data from the projected data
 - Step 9: Compute the reconstruction error

In [1]:
import numpy as np

In [93]:
class myPCA:
    def __init__(self):
        self.k = None
        self.components = None
        self.mean = None
        self.variance_share = None

    def find_k(self, X, threshold = 0.95):
        C = X.T @ X / X.shape[0]
        eigen_values, eigen_vectors = np.linalg.eigh(C)
        eigen_values = np.sort(eigen_values)[::-1]  # Sort and reverse in one step
        total = eigen_values.sum()
        for k in range(X.shape[0]):  # Corrected from x.shape[0] to X.shape[0]
            if eigen_values[:k+1].sum() / total >= threshold:  # Corrected from x to v
                return k + 1
        self.k = len(eigen_values)
        return self.k
    
    def fit(self, x, k = None):
        self.x = x
        if k is not None:
            self.k = k
        else:
            self.k = self.find_k(x)

        '''
        1. centre data
        2. compute covariance matrix
        3. compute eigenvalues and eigenvectors
        4. sort eigenvectors by decreasing eigenvalues
        5. choose k eigenvectors that correspond to k largest eigenvalues
        '''
        # centre data
        self.mean = np.mean(self.x , axis =0)
        x = self.x - self.mean

        # compute covariance matrix
        C = x.T @ x / x.shape[0]
        values, vectors = np.linalg.eigh(C)

        #sort eigenvectors by decreasing eigenvalues
        vectors = vectors[:,::-1]
        values = values[::-1]

        #store the first k eigenvectors
        self.components = vectors[:,:self.k]
        self.eigenvalues = values[:self.k]
        self.variance_share = np.sum(values[:self.k])/np.sum(values)



    def transform(self, x):
        '''
        6. Construct a new matrix with the k eigenvectors
        7. Project the original data onto the new matrix
        '''
        x = x - self.mean
        self.transformed = x @ self.components
        return self.transformed
    
    def reconstruct(self, x):
        '''
        8. Reconstruct the original data from the projected data
        '''
        self.reconstructed = self.transformed @ self.components.T + self.mean
        return self.reconstructed
    
    def reconstruction_error_(self, x):
        '''
        9. Compute the reconstruction error
        '''
        self.error = np.mean(np.linalg.norm(x - self.reconstructed, axis = 1)**2)
        return self.error
    
    def get_eigenvalues(self):
        return self.eigenvalues
    
    def get_eigenvectors(self):
        return self.components
    
    
        
        
    
    

## Example

In [13]:
x = np.array([
    [1.2, 3.4, 5.6, 7.8, 9.0, 1.1, 2.2, 3.3],
    [4.5, 5.6, 7.8, 8.9, 0.1, 1.2, 3.4, 4.5],
    [6.7, 8.9, 0.1, 1.2, 2.3, 4.5, 5.6, 6.7],
    [7.8, 9.0, 1.1, 2.2, 3.3, 5.6, 7.8, 8.9],
    [0.1, 1.2, 2.3, 4.5, 5.6, 7.8, 8.9, 9.0],
    [2.2, 3.3, 4.5, 6.7, 8.9, 0.1, 1.2, 2.3]
])

In [94]:
pca = myPCA()

In [95]:
pca.find_k(x, threshold = 0.95)

3

k value can be input in fit() function if needed. Otherwise algorithm calculates value based on threshold in find_k() function.

pca.fit(x, 4)

In [96]:
pca.fit(x )

In [97]:
pca.transform(x)

array([[ 7.20795568,  1.0775818 ,  1.46221582],
       [ 2.95092781, -5.22040934, -5.23836994],
       [-6.74567606, -2.54465505,  2.28244943],
       [-8.11365404, -1.33373033,  1.06554504],
       [-2.48764916,  8.06203586, -2.67503336],
       [ 7.18809577, -0.04082294,  3.103193  ]])

In [98]:
pca.variance_share

0.9829505724631614

In [99]:
pca.reconstruct(x)

array([[1.23672239, 2.80926566, 5.24606331, 7.47060784, 8.56090553,
        0.94293318, 2.02776315, 3.02786315],
       [4.5219922 , 5.67119191, 7.85704204, 8.95462403, 0.17556291,
        1.21509119, 3.43522666, 4.54861963],
       [7.41952297, 9.13845852, 0.64281078, 1.74791244, 3.10674112,
        4.45672685, 6.0717457 , 7.24969803],
       [7.06649351, 8.66886245, 0.49090746, 1.58925296, 2.40744471,
        5.62149953, 7.2904177 , 8.2957806 ],
       [0.21428667, 1.34789282, 2.45585925, 4.65223825, 5.81577853,
        7.82138487, 9.01075343, 9.14209066],
       [2.04098227, 3.76432863, 4.70731717, 6.88536447, 9.13356721,
        0.24236438, 1.26409335, 2.43594793]])

In [100]:
pca.reconstruction_error_(x)

1.1107559961687652

In [101]:
pca.get_eigenvectors()

array([[-0.30884117, -0.48132721,  0.15832499],
       [-0.31015877, -0.49851243,  0.23849484],
       [ 0.34981599, -0.17751574, -0.44505998],
       [ 0.40680508, -0.07906917, -0.40560929],
       [ 0.31724226,  0.4299002 ,  0.64581307],
       [-0.36112725,  0.3824365 , -0.17064343],
       [-0.3858341 ,  0.31152243, -0.25772709],
       [-0.37501143,  0.23222323, -0.20697627]])

In [102]:
pca.get_eigenvalues()


array([38.30920777, 16.94434179,  8.7848611 ])